In [ ]:
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import time

In [ ]:
# Chrome Brower와 Crome Driver Version 확인하기
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get('https://www.koreabaseball.com/Record/Crowd/GraphDaily.aspx')

In [ ]:
cloumnName, datetime, date, home ,away, stage, crowd = [], [], [], [], [], [], []

In [ ]:
# 자치구명 정보 가져오기
year_combobox = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_ddlSeason"]')
year_list = year_combobox.find_elements(By.TAG_NAME, 'option')
year_list
years = [option.get_attribute('value') for option in year_list][1:]
years

In [ ]:
gu_list_raw = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpRecord"]/table/thead')
gu_list_raw
gu_list = gu_list_raw.find_elements(By.TAG_NAME, 'tr')
gu_list
for row in gu_list:
    for detail in row.find_elements(By.TAG_NAME, 'th'):
        cloumnName.append(detail.text)

In [ ]:
def test():
    # 자치구명 정보 가져오기
    gu_list_raw = driver.find_element(By.XPATH, '//*[@id="cphContents_cphContents_cphContents_udpRecord"]/table/tbody')
    gu_list_raw
    gu_list = gu_list_raw.find_elements(By.TAG_NAME, 'tr')
    gu_list
    for row in gu_list:
        # for detail in row.find_elements(By.TAG_NAME, 'td'):
        #     print(detail.text)
        print(row.text.split())
        datetime.append(row.text.split()[0])
        date.append(row.text.split()[1])
        home.append(row.text.split()[2])
        away.append(row.text.split()[3])
        stage.append(row.text.split()[4])
        crowd.append(row.text.split()[5])

In [ ]:
# 조회버튼 click
for year in years:
    # print(int(year)-2020)
    #년도
    xpath = f'//*[@id="cphContents_cphContents_cphContents_ddlSeason"]/option[{int(year)-2020}]'
    element_sel_gu = driver.find_element(By.XPATH, xpath).click()
    # 소속구단
    xpath = '//*[@id="cphContents_cphContents_cphContents_ddlTeam"]/option[11]'
    element_sel_gu = driver.find_element(By.XPATH, xpath).click()
    # 홈 경기
    xpath = '//*[@id="cphContents_cphContents_cphContents_ddlHomeAway"]/option[2]'
    element_sel_gu = driver.find_element(By.XPATH, xpath).click()
    # 클릭
    xpath = '//*[@id="cphContents_cphContents_cphContents_btnSearch"]'
    element_sel_gu = driver.find_element(By.XPATH, xpath).click()

    time.sleep(1)
    test()
    

In [ ]:
df = pd.DataFrame(
    {
        '날짜' : datetime,
        '요일' : date,
        '홈팀' : home,
        '원정팀' : away,
        '장소' : stage,
        '관중 수' : crowd
    },
)
df

In [ ]:
def str_to_int(value):
    return int(value.replace(',', ''))

In [ ]:
df['관중 수'] = df['관중 수'].apply(str_to_int)
df['날짜'] = pd.to_datetime(df['날짜'])

In [ ]:
df

In [ ]:
df.to_csv('test.csv', index=False)